In [1]:
import os
import gc
import cv2
import time
import numpy as np  
np.set_printoptions(threshold=np.inf)
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

from keras import layers
from keras.models import Model
from keras.layers import Input,Dense,Flatten,Dropout,ZeroPadding2D,BatchNormalization,Activation,add,AveragePooling2D
from keras.layers.convolutional import Conv2D,MaxPooling2D  
from keras.applications.imagenet_utils import preprocess_input

Using TensorFlow backend.


In [2]:
def read_image(image_name):
    im=cv2.imread(image_name)
    im=cv2.resize(im, (227, 227))
    return im

#数据扩充
def img_Rotation(img,angel):
    if(0 == angel):
        dst = img
    else:
        rows,cols=img.shape[:2]
        #angel度旋转
        M=cv2.getRotationMatrix2D((cols/2,rows/2),angel,1)
        dst=cv2.warpAffine(img,M,(cols,rows))
    
    return dst

In [3]:
dataset = 'AID'
Extension='.jpg'
ModelName = "AlexNet"
batch_size = 64
epochs = 10

base_path='./Dataset/' + dataset

All_Labels = os.listdir(os.path.join(base_path, 'Train'))
num_classes = len(All_Labels)
images_Train = []
labels_Train = []
images_Val = []
labels_Val = []

for i in range(0, num_classes):
    file_dir = os.path.join(base_path, 'Train', All_Labels[i])
    file_names = os.listdir(file_dir)
    for file_name in file_names:
        if(file_name.endswith(Extension)):
            finalFileName = os.path.join(file_dir, file_name)
            Label = np.linspace(0, 0, num_classes, dtype='int32')
            Label[i] = 1
            original_img = read_image(finalFileName)
            flipped_img = cv2.flip(original_img, 0)
            
            for j in [0, 45, 90, 135, 180, 225, 270, 315]: 
                images_Train.append(img_Rotation(original_img, j))
                labels_Train.append(Label)               
                images_Train.append(img_Rotation(flipped_img, j))
                labels_Train.append(Label)
                
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + "Get %d Train Images" %(len(images_Train)))

print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + "np.array")
X_Train = np.array(images_Train, dtype='float32')
Y_Train = np.array(labels_Train)
del images_Train
gc.collect()

print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + "preprocess")
X_Train = preprocess_input(X_Train)
gc.collect()
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + "Done!")

2020-02-05 19:14:25Get 128000 Train Images
2020-02-05 19:14:25np.array
2020-02-05 20:10:59preprocess
2020-02-05 23:34:39Done!


In [4]:
from keras.optimizers import SGD  
def AlexNet(shape):
    inpt = Input(shape=shape)  
    x = Conv2D(96,(11,11),strides=(4,4),padding='valid',activation='relu',kernel_initializer='uniform')(inpt)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(3,3),strides=(2,2))(x)
    x = Conv2D(256,(5,5),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform')(x) 
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(3,3),strides=(2,2))(x)
    x = Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform')(x) 
    x = BatchNormalization()(x) 
    x = Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform')(x) 
    x = BatchNormalization()(x) 
    x = Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform')(x) 
    x = BatchNormalization()(x) 
    x = MaxPooling2D(pool_size=(3,3),strides=(2,2))(x)
    x = Flatten()(x)
    x = Dense(4096,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes,activation='softmax')(x)
    
    model = Model(inputs=inpt,outputs=x, name=ModelName)  
    sgd = SGD(decay=0.0001,momentum=0.9)  
    model.compile(loss='categorical_crossentropy',optimizer="SGD",metrics=['accuracy']) 
    
    return model

In [5]:
model = AlexNet(X_Train.shape[1:])
model.summary()

Model: "AlexNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 227, 227, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization_1 (Batch (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 256)       0   

In [10]:
history = model.fit(X_Train, Y_Train, batch_size=batch_size, epochs=epochs, shuffle=True)

Epoch 1/10
128000/128000 [==============================] - 4772s 37ms/step - loss: 0.0252 - accuracy: 0.9915
Epoch 2/10
128000/128000 [==============================] - 5997s 47ms/step - loss: 0.0224 - accuracy: 0.9925
Epoch 3/10
128000/128000 [==============================] - 5994s 47ms/step - loss: 0.0218 - accuracy: 0.9927
Epoch 4/10
128000/128000 [==============================] - 5934s 46ms/step - loss: 0.0215 - accuracy: 0.9929
Epoch 5/10
128000/128000 [==============================] - 9143s 71ms/step - loss: 0.0216 - accuracy: 0.9926
Epoch 6/10
128000/128000 [==============================] - 6969s 54ms/step - loss: 0.0168 - accuracy: 0.9944
Epoch 7/10
128000/128000 [==============================] - 6538s 51ms/step - loss: 0.0174 - accuracy: 0.9940
Epoch 8/10
128000/128000 [==============================] - 6373s 50ms/step - loss: 0.0172 - accuracy: 0.9943
Epoch 9/10
128000/128000 [==============================] - 6447s 50ms/step - loss: 0.0164 - accuracy: 0.9946
Epoch 10/1

In [9]:
save_folder = 'models/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + dataset
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# serialize model to JSON
import pickle
model_json = model.to_json()
with open(os.path.join(save_folder, ModelName + ".json"), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save(os.path.join(save_folder, ModelName + ".h5"))
#pickle.dump(history.history, open('history/UCMerced_LandUse/AlexNet.p','wb'))
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + "Saved Model to disk")

2020-02-09 08:59:34Saved Model to disk
